# Profile Earth Engine computation

The Earth Engine API provides tools for profiling the performance of your computations but they are not always the easiest to use to get the number you are looking for. The `geetools` library supercharge the original profiler to make any computation evaluation the easiest possible.

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/profile.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/profile.ipynb)

In [1]:
import ee, pytest_gee, os

if "EARTHENGINE_SERVICE_ACCOUNT" in os.environ:
    pytest_gee.init_ee_from_service_account()
elif "EARTHENGINE_PROJECT" in os.environ:
    pytest_gee.init_ee_from_token()
else:
    raise ValueError("Cannot authenticate with Earth Engine.")

## Set up environment

Install all the requireed libs if necessary. and perform the import satements upstream.

In [2]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [3]:
import ee
import geetools
import pandas as pd

In [4]:
# uncomment if authetication to GEE is needed
# ee.Authenticate()
# ee.Intialize(project="<your_project>")

## Example data

The following examples rely on a `ee.FeatureCollection` composed of three ecoregion features that define regions by which to reduce image data. The Image data are PRISM climate normals, where bands describe climate variables per month; e.g., July precipitation or January mean temperature.


In [5]:
ecoregions = (
    ee.FeatureCollection("projects/google/charts_feature_example")
    .select(["label", "value","warm"])
)

normClim = ee.ImageCollection('OREGONSTATE/PRISM/Norm91m').toBands()

## default profiler 

The default profiler from Earth Engine can be called as a context manager, it will print at the end of the cell the extensive description of your computation. 

In [6]:
with ee.profilePrinting():
    normClim.geetools.byBands(
        regions = ecoregions,
        reducer = "mean",
        scale = 500,
        regionId = "label",
        bands = [f"{i:02d}_tmean" for i in range(1,13)],
    ).getInfo()

 EECU·s PeakMem Count  Description
  0.258     59k     6  Algorithm Image.reduceRegions
  0.216    654k    87  Loading assets: projects/google/charts_feature_example
  0.126    345k   831  (plumbing)
  0.023    596k    86  no description available
  0.013    202k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/07@1662732032798195
  0.009    211k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/05@1662731334196830
  0.009    198k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/01@1662731626359925
  0.009    209k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/06@1662731651724226
  0.009    112k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/03@1662731338127317
  0.009    109k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/10@1662731228874571
  0.008    200k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/08@1662731245955723
  0.008    212k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/04@1662730567169297
  0.008    198k    11  Loading assets: OREGONSTATE/PRISM/Norm91m/12@1662

This result is extremely useful but cannot be further explored in the notebook.

## geetools profiler

The `geetools` profiler is a context manager object that fill a dictionary member (`profile`) with the content of the string profile. This dictionary can be transformed into a table easily.

In [7]:
# example with a simple function
with ee.Profiler() as p:
    ee.Number(3.14).add(0.00159).getInfo()
p.profile

{'EECU-s': [0.0, None],
 'PeakMem': [4720, 3200],
 'Count': [3, 3],
 'Description': ['(plumbing)', 'Algorithm Number.add']}

With a bigger method we can valorized the results as a pandas dataframe and extract key informations.

In [8]:
with ee.Profiler() as p:
    normClim.geetools.byBands(
        regions = ecoregions,
        reducer = "mean",
        scale = 500,
        regionId = "label",
        bands = [f"{i:02d}_tmean" for i in range(1,13)],
    ).getInfo()
df = pd.DataFrame(p.profile)
df.head()

,EECU-s,PeakMem,Count,Description
0,0.219,654000,87,Loading assets: projects/google/charts_feature...
1,0.192,59000,6,Algorithm Image.reduceRegions
2,0.125,363000,831,(plumbing)
3,0.021,607000,86,no description available
4,0.009,199000,11,Loading assets: OREGONSTATE/PRISM/Norm91m/09@1...


In [9]:
# total EECU cost of the computation
float(df["EECU-s"].sum())

KeyError: 'EECU·s'